# TODOs
- Level 0 RandomForest and LinearRegression are not giving good results
- Change to another models and Hypertune and then try running all levels and see teh final score

# Install packages

In [ ]:
! pip install optuna

     |████████████████████████████████| 308 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 68.7 MB/s 
     |████████████████████████████████| 80 kB 11.9 MB/s 
     |████████████████████████████████| 75 kB 5.7 MB/s 
     |████████████████████████████████| 112 kB 92.8 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
     |████████████████████████████████| 148 kB 96.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=ead39c4869c3759b43f4d3464966f96d5512a3ea4a2d76c382e68bccb999fd08
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 37 kB/s 


In [ ]:
! pip uninstall -y lightgbm
!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 1.5 MB 7.5 MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


# <font color='red'>Hyperparameter tuning of Level 0</font>


# Level 0 - Predictions and Test

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import optuna
import lightgbm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

Mounted at /content/drive


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


## XGBRegressor - Level 0

In [ ]:
xgb_params_basic = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        # Remember to increase n_estimators to 10,000 and increasing optuna trials to 200
        'n_estimators': 1000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters            
        }

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    # xgb_params_basic with early_stopping_rounds
    model = XGBRegressor(**xgb_params_basic, 
                         early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(f"Mean {np.mean(scores)} Std: {np.std(scores)}")

0 0.7179557457798484
1 0.7251265811151238
2 0.7211712857184023
3 0.720258568603719
4 0.715567264190993
Mean 0.7200158890816174 Std: 0.003210839816664595


In [ ]:
# xgb_params_basic and n_estimators=1000
# Mean 0.7200158890816174 Std: 0.003210839816664595

### Hyperparameter tuning using Optuna

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=1,
        objective='reg:squarederror',
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=10000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
# 50 trials for now, try with increasing it to 200
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=17, values=[0.7157276620593984], datetime_start=datetime.datetime(2021, 10, 31, 0, 13, 50, 693628), datetime_complete=datetime.datetime(2021, 10, 31, 0, 14, 28, 288950), params={'learning_rate': 0.031077184179118103, 'reg_lambda': 4.33448471426343, 'reg_alpha': 2.1560725588114122, 'subsample': 0.622537355727661, 'colsample_bytree': 0.1078346097784747, 'max_depth': 4}, distributions={'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'reg_lambda': LogUniformDistribution(high=100.0, low=1e-08), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'subsample': UniformDistribution(high=1.0, low=0.1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'max_depth': IntUniformDistribution(high=7, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE, value=None)

In [ ]:
study.best_params

{'colsample_bytree': 0.1078346097784747,
 'learning_rate': 0.031077184179118103,
 'max_depth': 4,
 'reg_alpha': 2.1560725588114122,
 'reg_lambda': 4.33448471426343,
 'subsample': 0.622537355727661}

In [ ]:
# number=17, values=[0.7157276620593984
# {'colsample_bytree': 0.1078346097784747,
#  'learning_rate': 0.031077184179118103,
#  'max_depth': 4,
#  'reg_alpha': 2.1560725588114122,
#  'reg_lambda': 4.33448471426343,
#  'subsample': 0.622537355727661}

In [ ]:
# FrozenTrial(number=97, values=[0.7154483966165949]
# {'colsample_bytree': 0.10150613849518197,
#  'learning_rate': 0.0170575651219112,
#  'max_depth': 6,
#  'reg_alpha': 42.407027899569876,
#  'reg_lambda': 0.15981415171625069,
#  'subsample': 0.4473641140360583}

In [ ]:
xgb_params_level0_two = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
         # Remember to increase n_estimators to 10000
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters   
       
        'colsample_bytree': 0.10150613849518197,
        'learning_rate': 0.0170575651219112,
        'max_depth': 6,
        'reg_alpha': 42.407027899569876,
        'reg_lambda': 0.15981415171625069,
        'subsample': 0.4473641140360583       
        }

xgb_params_level0_one = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
         # Remember to increase n_estimators to 10000
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters   
       
        'colsample_bytree': 0.1078346097784747,
        'learning_rate': 0.031077184179118103,
        'max_depth': 4,
        'reg_alpha': 2.1560725588114122,
        'reg_lambda': 4.33448471426343,
        'subsample': 0.622537355727661
        }

xgb_params_mahtab_new_two = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        # Remember to increase n_estimators to 10000
        'n_estimators': 10000, 

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters        
               
        'colsample_bytree': 0.10846937353161965,
        'learning_rate': 0.013677172081628455,
        'max_depth': 5,
        'reg_alpha': 23.99638282497684,
        'reg_lambda': 0.2054555005956808,
        'subsample': 0.8343156327805082
        }

### Generate Level 0 - Model 1 data

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    # model = XGBRegressor(**xgb_params_level0_one,
    #                      early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model = XGBRegressor(**xgb_params_mahtab_new_two,
                         early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model.fit(xtrain, ytrain)

 
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/test_pred_1.csv", index=False)

0 0.7154645629788147
1 0.722943748309661
2 0.7185910629892786
3 0.7181161409289041
4 0.7133596700601114
0.717695037053354 0.003234179412686518


In [ ]:
# xgb_params_level0_one with 1000 estimators - 0.7226109493317254 0.003223193537630209
# xgb_params_level0_one with 10,000 estimators - 0.7188470354299847 0.003365272059686831

# Level 0 - BEST SO FAR
# xgb_params_mahtab_new_two with 10,000 estimators - 0.717695037053354 0.003234179412686518

## LGBMRegressor - Level 0

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
import lightgbm
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

Mounted at /content/drive


In [ ]:
lgbm_params_basic = {
        'random_state': 1,
        # Remember to increase n_estimators to 10000
        'n_estimators': 500,              
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,              
        'metric': 'l2',
        'min_child_weight': 100.0,              
        'reg_alpha': 10.0,
        'reg_lambda': 0.1,
        }

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    # lgbm_params_basic with early_stopping_rounds
    model = LGBMRegressor(**lgbm_params_basic)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True)] )

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(f"Mean {np.mean(scores)} Std: {np.std(scores)}")

0 0.7189164631405961
1 0.72659840122427
2 0.7220306458844435
3 0.7212566259625696
4 0.7169868127616933
Mean 0.7211577897947146 Std: 0.0032493843616696145


In [ ]:
# lgbm_params_basic and n_estimators=500 
# Mean 0.7211577897947146 Std: 0.0032493843616696145

### Hyperparameter tuning using Optuna

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0

    reg_alpha = trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    reg_lambda = trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    subsample = trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
    learning_rate = trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
    max_depth = trial.suggest_categorical('max_depth', [10,20,100]),
    num_leaves = trial.suggest_int('num_leaves', 20, 3000),
    min_child_samples = trial.suggest_int('min_child_samples', 1, 300),
       
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = LGBMRegressor(
        random_state = 1,
        metric = 'rmse', 
        n_estimators = 10000,
        
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        colsample_bytree = colsample_bytree,
        subsample = subsample,
        learning_rate = learning_rate,
        max_depth = max_depth,
        num_leaves = num_leaves,
        min_child_samples = min_child_samples      
    )
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True), 
                         lightgbm.early_stopping(300)])
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=42, values=[0.7170991148991163], datetime_start=datetime.datetime(2021, 10, 31, 5, 25, 23, 283200), datetime_complete=datetime.datetime(2021, 10, 31, 5, 28, 54, 54688), params={'reg_alpha': 0.004698521384225176, 'reg_lambda': 0.003334404680712291, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.006, 'max_depth': 10, 'num_leaves': 1343, 'min_child_samples': 299}, distributions={'reg_alpha': LogUniformDistribution(high=10.0, low=0.001), 'reg_lambda': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'learning_rate': CategoricalDistribution(choices=(0.006, 0.008, 0.01, 0.014, 0.017, 0.02)), 'max_depth': CategoricalDistribution(choices=(10, 20, 100)), 'num_leaves': IntUniformDistribution(high=3000, low=20, step=1), 'min_child_samples': IntUniformDistribution(high=300, low=1, step=1)}, user_a

In [ ]:
study.best_params

{'colsample_bytree': 0.3,
 'learning_rate': 0.006,
 'max_depth': 10,
 'min_child_samples': 299,
 'num_leaves': 1343,
 'reg_alpha': 0.004698521384225176,
 'reg_lambda': 0.003334404680712291,
 'subsample': 0.8}

In [ ]:
# FrozenTrial(number=42, values=[0.7170991148991163], datetime_start=datetime.datetime(2021, 10, 31, 5, 25, 23, 283200), datetime_complete=datetime.datetime(2021, 10, 31, 5, 28, 54, 54688), params={'reg_alpha': 0.004698521384225176, 'reg_lambda': 0.003334404680712291, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.006, 'max_depth': 10, 'num_leaves': 1343, 'min_child_samples': 299}, distributions={'reg_alpha': LogUniformDistribution(high=10.0, low=0.001), 'reg_lambda': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'learning_rate': CategoricalDistribution(choices=(0.006, 0.008, 0.01, 0.014, 0.017, 0.02)), 'max_depth': CategoricalDistribution(choices=(10, 20, 100)), 'num_leaves': IntUniformDistribution(high=3000, low=20, step=1), 'min_child_samples': IntUniformDistribution(high=300, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=42, state=TrialState.COMPLETE, value=None)
# {'colsample_bytree': 0.3,
#  'learning_rate': 0.006,
#  'max_depth': 10,
#  'min_child_samples': 299,
#  'num_leaves': 1343,
#  'reg_alpha': 0.004698521384225176,
#  'reg_lambda': 0.003334404680712291,
#  'subsample': 0.8}

In [ ]:
# FrozenTrial(number=24, values=[0.7168448540125021]
# {'colsample_bytree': 0.3,
#  'learning_rate': 0.02,
#  'max_depth': 20,
#  'min_child_samples': 110,
#  'min_data_per_groups': 26,
#  'num_leaves': 33,
#  'reg_alpha': 0.001720365664958218,
#  'reg_lambda': 0.32492525826490454,
#  'subsample': 0.7}

In [ ]:
lgb_params_level0_two = {
        'random_state': 1,
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        # Remember to increase n_estimators to 10000
        "n_estimators": 10000,
        "early_stopping_round": 300,
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
 
        'colsample_bytree': 0.3,
        'learning_rate': 0.006,
        'max_depth': 10,
        'min_child_samples': 299,
        'num_leaves': 1343,
        'reg_alpha': 0.004698521384225176,
        'reg_lambda': 0.003334404680712291,
        'subsample': 0.8
 }

lgb_params_level0_one = {
        'random_state': 1,
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        # Remember to increase n_estimators to 10000
        "n_estimators": 10000,
        "early_stopping_round": 300,
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,

        'colsample_bytree': 0.3,
        'learning_rate': 0.02,
        'max_depth': 20,
        'min_child_samples': 110,
        'min_data_per_groups': 26,
        'num_leaves': 33,
        'reg_alpha': 0.001720365664958218,
        'reg_lambda': 0.32492525826490454,
        'subsample': 0.7
      }

lgb_params_one = {
        'random_state': 1,
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        # Remember to increase n_estimators to 10000
        "n_estimators": 10000,
        "early_stopping_round": 300,
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,

        'lambda_l1': 0.00472279780583036, 
        'lambda_l2': 2.9095205689488508e-05, 
        'num_leaves': 158, 
        'feature_fraction': 0.7386878356648194, 
        'bagging_fraction': 0.8459744550725283, 
        'bagging_freq': 2, 
        'max_depth': 2, 
        'max_bin': 249, 
        'learning_rate': 0.044738463593017294,
        'min_child_samples': 13,
      }

### Generate Level 0 - Model 2 data

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    # model = LGBMRegressor(**lgb_params_level0_one)
    model = LGBMRegressor(**lgb_params_level0_two)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True)] )
     
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/test_pred_2.csv", index=False)

[1000]	valid_0's rmse: 0.721277
[2000]	valid_0's rmse: 0.718186
[3000]	valid_0's rmse: 0.717393
[4000]	valid_0's rmse: 0.717097
0 0.717080735112011
[1000]	valid_0's rmse: 0.729041
[2000]	valid_0's rmse: 0.725954
[3000]	valid_0's rmse: 0.725117
[4000]	valid_0's rmse: 0.724868
1 0.7248524228849832
[1000]	valid_0's rmse: 0.724389
[2000]	valid_0's rmse: 0.721408
[3000]	valid_0's rmse: 0.72053
[4000]	valid_0's rmse: 0.720213
2 0.7201724767247922
[1000]	valid_0's rmse: 0.723349
[2000]	valid_0's rmse: 0.720485
[3000]	valid_0's rmse: 0.719796
[4000]	valid_0's rmse: 0.719592
3 0.7195616127364113
[1000]	valid_0's rmse: 0.719406
[2000]	valid_0's rmse: 0.716494
[3000]	valid_0's rmse: 0.715687
[4000]	valid_0's rmse: 0.715446
4 0.7153912192530189
0.7194116933422434 0.0032187072967378035


In [ ]:
# lgbm_params_basic with 500 estimators - 0.7211577897947146 Std: 0.0032493843616696145
# lgb_params_level0_one with 10,000 estimators - 0.7192859619906993 0.0033222193803162147
# lgb_params_level0_two with 10,000 estimators - 0.7194116933422434 0.0032187072967378035

# Level 0 - BEST SO FAR

## CatBoostRegressor - Level 0

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import optuna
import lightgbm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

Mounted at /content/drive


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


In [ ]:
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [ ]:
catb_params_basic = {      
        "verbose": False, 
        "use_best_model": True,        
      }

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
 
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])

    model = CatBoostRegressor(**catb_params_basic)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=300)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7182498184556952
1 0.7258754490648508
2 0.7216273751079832
3 0.7206636973643382
4 0.7162692142289554
0.7205371108443647 0.0032603436401383453


In [ ]:
# catb_params_basic - Level 0
# 0 0.7182498184556952
# 1 0.7258754490648508
# 2 0.7216273751079832
# 3 0.7206636973643382
# 4 0.7162692142289554
# 0.7205371108443647 0.0032603436401383453

### Hyperparameter tuning using Optuna

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    learning_rate = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
    depth = trial.suggest_int('depth', 9, 15)
    l2_leaf_reg = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    min_child_samples = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
          
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = CatBoostRegressor(
        random_state = 1,
        # Changed to 2000 from 10,000
        iterations = 2000,
        eval_metric = 'RMSE',
        use_best_model = True,
        od_type = 'iter',
        od_wait = 20,
        grow_policy = 'Depthwise',
        logging_level = 'Silent',
        
        learning_rate = learning_rate,
        depth = depth,
        l2_leaf_reg = l2_leaf_reg,
        min_child_samples = min_child_samples       
    )
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=300)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=36, values=[0.7183912022061714], datetime_start=datetime.datetime(2021, 10, 31, 14, 45, 22, 980643), datetime_complete=datetime.datetime(2021, 10, 31, 14, 49, 7, 216831), params={'learning_rate': 0.017, 'depth': 9, 'l2_leaf_reg': 5.5, 'min_child_samples': 4}, distributions={'learning_rate': DiscreteUniformDistribution(high=0.02, low=0.001, q=0.001), 'depth': IntUniformDistribution(high=15, low=9, step=1), 'l2_leaf_reg': DiscreteUniformDistribution(high=5.5, low=1.0, q=0.5), 'min_child_samples': CategoricalDistribution(choices=(1, 4, 8, 16, 32))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=36, state=TrialState.COMPLETE, value=None)

In [ ]:
study.best_params

{'depth': 9,
 'l2_leaf_reg': 5.5,
 'learning_rate': 0.017,
 'min_child_samples': 4}

In [ ]:
# FrozenTrial(number=36, values=[0.7183912022061714], datetime_start=datetime.datetime(2021, 10, 31, 14, 45, 22, 980643), datetime_complete=datetime.datetime(2021, 10, 31, 14, 49, 7, 216831), params={'learning_rate': 0.017, 'depth': 9, 'l2_leaf_reg': 5.5, 'min_child_samples': 4}, distributions={'learning_rate': DiscreteUniformDistribution(high=0.02, low=0.001, q=0.001), 'depth': IntUniformDistribution(high=15, low=9, step=1), 'l2_leaf_reg': DiscreteUniformDistribution(high=5.5, low=1.0, q=0.5), 'min_child_samples': CategoricalDistribution(choices=(1, 4, 8, 16, 32))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=36, state=TrialState.COMPLETE, value=None)
# {'depth': 9,
#  'l2_leaf_reg': 5.5,
#  'learning_rate': 0.017,
#  'min_child_samples': 4}

In [ ]:
# Past run
# {'depth': 9,
#  'l2_leaf_reg': 5.5,
#  'learning_rate': 0.005,
#  'min_child_samples': 32}

In [ ]:
catb_params_level0_one = {      
      'random_state' : 1,
       # Remember to increase n_estimators to 10000
      'iterations' : 10000,
      'eval_metric' : 'RMSE',
      'use_best_model' : True,
      'od_type' : 'iter',
      'od_wait' : 20,
      'grow_policy' : 'Depthwise',
      'logging_level' : 'Silent',
 
      'depth': 9,
      'l2_leaf_reg': 5.5,
      'learning_rate': 0.017,
      'min_child_samples': 4
 }


### Generate Level 0 - Model 3 data

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = CatBoostRegressor(**catb_params_level0_one)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=1000)
     
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/test_pred_3.csv", index=False)

0 0.7183466623151733
1 0.7266246070349631
2 0.7214689216405634
3 0.7212982927977727
4 0.7165263145916348
0.7208529596760215 0.0034319390470785058


In [ ]:
# catb_params_basic - 0.7174672048416613 0.003167382534210746
# catb_params_level1_one - 0.7208529596760215 0.0034319390470785058

## RandomForestRegressor

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import optuna
import lightgbm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

Mounted at /content/drive


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


In [ ]:
rfr_params_basic = {
        'random_state': 1,
        'n_jobs': -1,
        'warm_start' : True,
        # Remember to increase n_estimators to 10000 and increasing trials to 200
        'n_estimators': 1000,
        'max_depth' : 3,
        }

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = RandomForestRegressor(**rfr_params_basic)                         
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(f"Mean {np.mean(scores)} Std: {np.std(scores)}")

In [ ]:
# 0 0.7392003930152541
# 1 0.7463703011460007
# 2 0.7422826672013653
# 3 0.7408205353981845
# 4 0.7363088915653575
# Mean 0.7409965576652324 Std: 0.0033395389348913453

### Hyperparameter tuning using Optuna

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    # Changed n_estimators upper limit to 1000 from 10000
    # Note that min_samples_split and min_samples_leaf if int should be > 1
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 4, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 150)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 60) 

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = RandomForestRegressor(
        random_state=1,
        n_jobs = -1,
        warm_start = True,           
        
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,   
    )
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
# 50 trials for now, try with increasing it to 100
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

In [ ]:
study.best_params

In [ ]:
# FrozenTrial(number=33, values=[0.7286918572081191], datetime_start=datetime.datetime(2021, 11, 3, 18, 35, 34, 308479), datetime_complete=datetime.datetime(2021, 11, 3, 18, 57, 7, 82292), params={'n_estimators': 825, 'max_depth': 38, 'min_samples_split': 64, 'min_samples_leaf': 10}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=100, step=1), 'max_depth': IntUniformDistribution(high=50, low=4, step=1), 'min_samples_split': IntUniformDistribution(high=150, low=2, step=1), 'min_samples_leaf': IntUniformDistribution(high=60, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=33, state=TrialState.COMPLETE, value=None)
# [I 2021-11-03 18:57:07,082] Trial 33 finished with value: 0.7286918572081191 and parameters: {'n_estimators': 825, 'max_depth': 38, 'min_samples_split': 64, 'min_samples_leaf': 10}. Best is trial 33 with value: 0.7286918572081191.
# {'max_depth': 38,
#  'min_samples_leaf': 10,
#  'min_samples_split': 64,
#  'n_estimators': 825}

In [ ]:
rfr_params_level0_one = {
        'random_state': 1,
        'n_jobs': -1,
        'warm_start' : True,
        # Remember to increase n_estimators to 5000
        # 'n_estimators': 2000,
        'n_estimators': 825,      
        'max_depth': 38,
        'min_samples_leaf': 10,
        'min_samples_split': 64,
        }


### Generate Level 0 - Model 4 data

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = RandomForestRegressor(**rfr_params_level0_one)
    model.fit(xtrain, ytrain)
 
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_4"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/train_pred_4.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_4"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/test_pred_4.csv", index=False)

0 0.7286918572081191
1 0.7363309978395016
2 0.7318213983698808
3 0.7304895811712103
4 0.7263006544337741
0.7307268978044972 0.0033583689041013064


In [ ]:
# xgb_params_level0_one with 10,000 estimators - 0.7188470354299847 0.003365272059686831

# rfr_params_level0_one with 825 estimators - 0.7307268978044972 0.0033583689041013064

## LinearRegression

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import optuna
import lightgbm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


In [ ]:
lr_params_basic = {
         'n_jobs': -1,              
        }

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

# 3 degree polynomial
poly = preprocessing.PolynomialFeatures(degree=3, interaction_only=True, include_bias=False)
train_poly = poly.fit_transform(df[numerical_cols])
test_poly = poly.fit_transform(df_test[numerical_cols])

df_poly = pd.DataFrame(train_poly, columns=[f"poly_{i}" for i in range(train_poly.shape[1])])
df_test_poly = pd.DataFrame(test_poly, columns=[f"poly_{i}" for i in range(test_poly.shape[1])])

df = pd.concat([df, df_poly], axis=1)
df_test = pd.concat([df_test, df_test_poly], axis=1)

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = LinearRegression(**lr_params_basic)
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(f"Mean {np.mean(scores)} Std: {np.std(scores)}")

In [ ]:
# lr_params_basic
# 0.7393145513167105 Std: 0.0035716236440714805

### Generate Level 0 - Model 5 data

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

# 3 degree polynomial
poly = preprocessing.PolynomialFeatures(degree=3, interaction_only=True, include_bias=False)
train_poly = poly.fit_transform(df[numerical_cols])
test_poly = poly.fit_transform(df_test[numerical_cols])

df_poly = pd.DataFrame(train_poly, columns=[f"poly_{i}" for i in range(train_poly.shape[1])])
df_test_poly = pd.DataFrame(test_poly, columns=[f"poly_{i}" for i in range(test_poly.shape[1])])

df = pd.concat([df, df_poly], axis=1)
df_test = pd.concat([df_test, df_test_poly], axis=1)

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = LinearRegression(**lr_params_basic)
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(f"Mean {np.mean(scores)} Std: {np.std(scores)}")

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_5"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/train_pred_5.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_5"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/test_pred_5.csv", index=False)

In [ ]:
# lr_params_basic - Mean 0.7396145142808443 Std: 0.0033216489483374797


# Level 1 - Predictions and Test

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import optuna
import lightgbm

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


## XGBRegressor

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

# Valid predictions
df1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_1.csv")
df2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_2.csv")
df3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_3.csv")
df4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_4.csv")
df5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_5.csv")

# Test predictions
df_test1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_1.csv")
df_test2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_2.csv")
df_test3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_3.csv")
df_test4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_4.csv")
df_test5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_5.csv")

Mounted at /content/drive


In [ ]:
# Get Valid predictions in df
df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")

# Get Test predictions in df_test
df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,pred_1,pred_2,pred_3,pred_4,pred_5
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702,8.077594,8.048778,8.043311,7.981364,8.070465
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940,8.311259,8.322672,8.336907,8.270948,8.193022
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099,8.416652,8.394490,8.383222,8.364884,8.328782
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372,8.458765,8.427877,8.446128,8.344412,8.157117
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412,8.191111,8.150499,8.158664,8.167140,8.194156


In [ ]:
xgb_params_basic = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        # Remember to increase n_estimators to 1000 and increasing trials to 200
        'n_estimators': 1000,
        
        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters            
        }

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
 
    model = XGBRegressor(**xgb_params_basic, 
                         early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7162587079042024
1 0.7241757045674065
2 0.7196122190385031
3 0.7189579838470335
4 0.7141119983975416
0.7186233227509374 0.003401832110719174


In [ ]:
# With pred_1, 2, 3
# 0.7184428154785415 0.0031525023940106992

# With pred_1, 2, 3, 4, 5
# 0.7186233227509374 0.003401832110719174

In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


### Optuna Hyperameter Tuning

In [ ]:
#useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
# Just taking the 5 preds
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
object_cols = [col for col in useful_features if 'cat' in col]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=1,
        objective='reg:squarederror',
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=10000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
# 50 trials for now, try with increasing it to 200
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=37, values=[0.715194305497053], datetime_start=datetime.datetime(2021, 11, 16, 7, 26, 56, 90696), datetime_complete=datetime.datetime(2021, 11, 16, 7, 26, 57, 659124), params={'learning_rate': 0.036444530508175095, 'reg_lambda': 3.476365489904126e-06, 'reg_alpha': 5.092222000312598e-08, 'subsample': 0.7239859346173412, 'colsample_bytree': 0.8425801312700381, 'max_depth': 4}, distributions={'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'reg_lambda': LogUniformDistribution(high=100.0, low=1e-08), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'subsample': UniformDistribution(high=1.0, low=0.1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'max_depth': IntUniformDistribution(high=7, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=37, state=TrialState.COMPLETE, value=None)

In [ ]:
study.best_params

{'colsample_bytree': 0.8425801312700381,
 'learning_rate': 0.036444530508175095,
 'max_depth': 4,
 'reg_alpha': 5.092222000312598e-08,
 'reg_lambda': 3.476365489904126e-06,
 'subsample': 0.7239859346173412}

In [ ]:
# FrozenTrial(number=33, values=[0.7151575868340049], datetime_start=datetime.datetime(2021, 11, 8, 10, 33, 18, 473445), datetime_complete=datetime.datetime(2021, 11, 8, 10, 33, 20, 171980), params={'learning_rate': 0.05499560682522413, 'reg_lambda': 8.860360143553858e-08, 'reg_alpha': 0.8523626298620842, 'subsample': 0.27795736085350486, 'colsample_bytree': 0.7259974470114723, 'max_depth': 1}, distributions={'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'reg_lambda': LogUniformDistribution(high=100.0, low=1e-08), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'subsample': UniformDistribution(high=1.0, low=0.1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'max_depth': IntUniformDistribution(high=7, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=33, state=TrialState.COMPLETE, value=None)
# {'colsample_bytree': 0.7259974470114723,
#  'learning_rate': 0.05499560682522413,
#  'max_depth': 1,
#  'reg_alpha': 0.8523626298620842,
#  'reg_lambda': 8.860360143553858e-08,
#  'subsample': 0.27795736085350486}

In [ ]:
xgb_params_level1_one = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters            
        
        'colsample_bytree': 0.7259974470114723,
        'learning_rate': 0.05499560682522413,
        'max_depth': 1,
        'reg_alpha': 0.8523626298620842,
        'reg_lambda': 8.860360143553858e-08,
        'subsample': 0.27795736085350486
 }
 

xgb_params_level1_two = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters            
        
        'colsample_bytree': 0.8425801312700381,
        'learning_rate': 0.036444530508175095,
        'max_depth': 4,
        'reg_alpha': 5.092222000312598e-08,
        'reg_lambda': 3.476365489904126e-06,
        'subsample': 0.7239859346173412
      }

In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = XGBRegressor(**xgb_params_level1_one,
                          early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    # model = XGBRegressor(**xgb_params_level1_two,
    #                        early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    
    model.fit(xtrain, ytrain)
        
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_1.csv", index=False)

0 0.7154509818931295
1 0.7232419112890471
2 0.7184771412347586
3 0.7180315936858023
4 0.7134235748290576
0.7177250405863591 0.0033096659390267516


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [ ]:
# With pred_1, 2, 3
# xgb_params_level1_one - 0.7176203587258627 0.0031757533803808097
# xgb_params_level1_two - 0.7174662580595934 0.0031812557051354075

# xgb_params_level1_two - With pred_1, 2, 3, 4, 5
# 0.7247477328242711 0.0033270680988922233

# xgb_params_level1_one - With pred_1, 2, 3, 4, 5
# 0.7177250405863591 0.0033096659390267516

## LGBMRegressor

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

# Valid predictions
df1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_1.csv")
df2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_2.csv")
df3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_3.csv")
df4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_4.csv")
df5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_5.csv")

# Test predictions
df_test1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_1.csv")
df_test2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_2.csv")
df_test3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_3.csv")
df_test4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_4.csv")
df_test5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_5.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Get Valid predictions in df
df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")

# Get Test predictions in df_test
df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,pred_1,pred_2,pred_3,pred_4,pred_5
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702,8.077594,8.048778,8.043311,7.981364,8.070465
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940,8.311259,8.322672,8.336907,8.270948,8.193022
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099,8.416652,8.394490,8.383222,8.364884,8.328782
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372,8.458765,8.427877,8.446128,8.344412,8.157117
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412,8.191111,8.150499,8.158664,8.167140,8.194156


In [ ]:
lgbm_params_basic = {
        'random_state': 1,
        'n_estimators': 500,              
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,              
        'metric': 'l2',
        'min_child_weight': 100.0,              
        'reg_alpha': 10.0,
        'reg_lambda': 0.1,
        }

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
 
    model = LGBMRegressor(**lgbm_params_basic)   
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True)] )
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7160245036159335
1 0.7236712418357616
2 0.7191584602834706
3 0.7186986713617107
4 0.7139023018957124
0.7182910357985178 0.003294339460255852


In [ ]:
# With pred_1, 2, 3
# 0.718256894783831 0.0031629805047084154

# With pred_1, 2, 3, 4, 5
# 0.7182910357985178 0.003294339460255852

### Optuna Hyperameter Tuning

In [ ]:
# Just taking the 5 preds
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    reg_alpha = trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    reg_lambda = trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    subsample = trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
    learning_rate = trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
    max_depth = trial.suggest_categorical('max_depth', [10,20,100]),
    num_leaves = trial.suggest_int('num_leaves', 1, 1000),
    min_child_samples = trial.suggest_int('min_child_samples', 1, 300),
    cat_smooth = trial.suggest_int('min_data_per_groups', 1, 100)
   
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    model = LGBMRegressor(
        random_state = 1,
        metric = 'rmse', 
        n_estimators = 10000,
        
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        colsample_bytree = colsample_bytree,
        subsample = subsample,
        learning_rate = learning_rate,
        max_depth = max_depth,
        num_leaves = num_leaves,
        min_child_samples = min_child_samples,
        cat_smooth = cat_smooth
    )
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True), 
                         lightgbm.early_stopping(300)])
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=41, values=[0.7152385490213837], datetime_start=datetime.datetime(2021, 11, 16, 7, 53, 9, 362534), datetime_complete=datetime.datetime(2021, 11, 16, 7, 53, 40, 816438), params={'reg_alpha': 0.2564941911129974, 'reg_lambda': 0.026743760913443694, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.014, 'max_depth': 10, 'num_leaves': 2, 'min_child_samples': 135, 'min_data_per_groups': 64}, distributions={'reg_alpha': LogUniformDistribution(high=10.0, low=0.001), 'reg_lambda': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'learning_rate': CategoricalDistribution(choices=(0.006, 0.008, 0.01, 0.014, 0.017, 0.02)), 'max_depth': CategoricalDistribution(choices=(10, 20, 100)), 'num_leaves': IntUniformDistribution(high=1000, low=1, step=1), 'min_child_samples': IntUniformDistribution(high=300, lo

In [ ]:
study.best_params

{'colsample_bytree': 0.8,
 'learning_rate': 0.014,
 'max_depth': 10,
 'min_child_samples': 135,
 'min_data_per_groups': 64,
 'num_leaves': 2,
 'reg_alpha': 0.2564941911129974,
 'reg_lambda': 0.026743760913443694,
 'subsample': 0.4}

In [ ]:
# FrozenTrial(number=41, values=[0.7152385490213837], datetime_start=datetime.datetime(2021, 11, 16, 7, 53, 9, 362534), datetime_complete=datetime.datetime(2021, 11, 16, 7, 53, 40, 816438), params={'reg_alpha': 0.2564941911129974, 'reg_lambda': 0.026743760913443694, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.014, 'max_depth': 10, 'num_leaves': 2, 'min_child_samples': 135, 'min_data_per_groups': 64}, distributions={'reg_alpha': LogUniformDistribution(high=10.0, low=0.001), 'reg_lambda': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'learning_rate': CategoricalDistribution(choices=(0.006, 0.008, 0.01, 0.014, 0.017, 0.02)), 'max_depth': CategoricalDistribution(choices=(10, 20, 100)), 'num_leaves': IntUniformDistribution(high=1000, low=1, step=1), 'min_child_samples': IntUniformDistribution(high=300, low=1, step=1), 'min_data_per_groups': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=41, state=TrialState.COMPLETE, value=None)
# {'colsample_bytree': 0.8,
#  'learning_rate': 0.014,
#  'max_depth': 10,
#  'min_child_samples': 135,
#  'min_data_per_groups': 64,
#  'num_leaves': 2,
#  'reg_alpha': 0.2564941911129974,
#  'reg_lambda': 0.026743760913443694,
#  'subsample': 0.4}

In [ ]:
lgbm_params_level1_one = {      
        'random_state': 1,
        'metric':'rmse',
        'n_estimators': 10000,

        'colsample_bytree': 0.8,
        'learning_rate': 0.014,
        'max_depth': 10,
        'min_child_samples': 135,
        'min_data_per_groups': 64,
        'num_leaves': 2,
        'reg_alpha': 0.2564941911129974,
        'reg_lambda': 0.026743760913443694,
        'subsample': 0.4
        }


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
  
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
   
    model = LGBMRegressor(**lgbm_params_level1_one)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              callbacks=[lightgbm.log_evaluation(period=1000, show_stdv=True), 
                         lightgbm.early_stopping(300)])
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
 
print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_2.csv", index=False)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.715283
[2000]	valid_0's rmse: 0.715252
[3000]	valid_0's rmse: 0.715246
[4000]	valid_0's rmse: 0.715243
[5000]	valid_0's rmse: 0.715241
[6000]	valid_0's rmse: 0.715239
Early stopping, best iteration is:
[6213]	valid_0's rmse: 0.715239
0 0.7152385490213837
[LightGBM] [Warning] Unknown parameter: min_data_per_groups
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.722783
[2000]	valid_0's rmse: 0.722764
Early stopping, best iteration is:
[1934]	valid_0's rmse: 0.722763
1 0.7227631691432711
[LightGBM] [Warning] Unknown parameter: min_data_per_groups
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.718391
[2000]	valid_0's rmse: 0.718371
[3000]	valid_0's rmse: 0.718369
Early stopping, best iteration is:
[3169]	valid_0's rmse: 0.718369
2 0.7183686946778719
[LightGBM] [Warnin

In [ ]:
# lgbm_params_basic - With pred_1, 2, 3 - 0.718256894783831 0.0031629805047084154
# lgbm_params_level1_one - With pred_1, 2, 3 - 0.7174507167082399 0.003172362451758317

# lgbm_params_level1_one - With pred_1, 2, 3, 4, 5 - 0.7174833303520474 0.003221485724817367

## CatBoostRegressor

In [ ]:
!pip install catboost

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

# Valid predictions
df1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_1.csv")
df2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_2.csv")
df3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_3.csv")
df4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_4.csv")
df5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_pred_5.csv")

# Test predictions
df_test1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_1.csv")
df_test2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_2.csv")
df_test3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_3.csv")
df_test4 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_4.csv")
df_test5 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test_pred_5.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Get Valid predictions in df
df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")
df = df.merge(df5, on="id", how="left")

# Get Test predictions in df_test
df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")
df_test = df_test.merge(df_test5, on="id", how="left")

In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,pred_1,pred_2,pred_3,pred_4,pred_5
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702,8.077594,8.048778,8.043311,7.981364,8.070465
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940,8.311259,8.322672,8.336907,8.270948,8.193022
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099,8.416652,8.394490,8.383222,8.364884,8.328782
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372,8.458765,8.427877,8.446128,8.344412,8.157117
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412,8.191111,8.150499,8.158664,8.167140,8.194156


In [ ]:
catb_params_basic = {      
        "verbose": False, 
        "use_best_model": True,        
      }

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
 
    model = CatBoostRegressor(**catb_params_basic)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7153451674069043
1 0.722800880152827
2 0.7183039636845021
3 0.717847345989088
4 0.7132271563315188
0.7175049027129681 0.0032176610627393147


In [ ]:
# catb_params_basic - With pred_1, 2, 3 - 0.7174672048416613 0.003167382534210746
# catb_params_basic - With pred_1, 2, 3, 4, 5,  - 0.7175049027129681 0.0032176610627393147

### Optuna Hyperameter Tuning

In [ ]:
# Just taking the 5 preds
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    learning_rate = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
    depth = trial.suggest_int('depth', 9, 15)
    l2_leaf_reg = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    min_child_samples = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
          
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    model = CatBoostRegressor(
        random_state = 1,
        iterations = 10000,
        eval_metric = 'RMSE',
        use_best_model = True,
        od_type = 'iter',
        od_wait = 20,
        grow_policy = 'Depthwise',
        logging_level = 'Silent',
        
        learning_rate = learning_rate,
        depth = depth,
        l2_leaf_reg = l2_leaf_reg,
        min_child_samples = min_child_samples       
    )
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=50)

In [ ]:
study.best_trial

FrozenTrial(number=19, values=[0.7153175806447779], datetime_start=datetime.datetime(2021, 11, 16, 8, 50, 25, 322897), datetime_complete=datetime.datetime(2021, 11, 16, 8, 51, 30, 630945), params={'learning_rate': 0.014000000000000002, 'depth': 9, 'l2_leaf_reg': 3.5, 'min_child_samples': 4}, distributions={'learning_rate': DiscreteUniformDistribution(high=0.02, low=0.001, q=0.001), 'depth': IntUniformDistribution(high=15, low=9, step=1), 'l2_leaf_reg': DiscreteUniformDistribution(high=5.5, low=1.0, q=0.5), 'min_child_samples': CategoricalDistribution(choices=(1, 4, 8, 16, 32))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=19, state=TrialState.COMPLETE, value=None)

In [ ]:
study.best_params

{'depth': 9,
 'l2_leaf_reg': 3.5,
 'learning_rate': 0.014000000000000002,
 'min_child_samples': 4}

In [ ]:
# {'depth': 9,
#  'l2_leaf_reg': 3.5,
#  'learning_rate': 0.014000000000000002,
#  'min_child_samples': 4}

In [ ]:
catb_params_level1_one = {      
      'random_state' : 1,
      'iterations' : 10000,
      'eval_metric' : 'RMSE',
      'use_best_model' : True,
      'od_type' : 'iter',
      'od_wait' : 20,
      'grow_policy' : 'Depthwise',
      'logging_level' : 'Silent',
 
      'depth': 9,
      'l2_leaf_reg': 3.5,
      'learning_rate': 0.014000000000000002,
      'min_child_samples': 4
 }


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4,pred_5
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.439672,8.416726,8.442819,8.231774,8.192121
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.386412,8.360824,8.315855,8.297833,8.414298
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.150599,8.227866,8.176430,8.088723,8.145324
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.391289,8.336769,8.332540,8.305391,8.302117
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.240445,8.229588,8.284710,8.320177,8.404584


In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4", "pred_5"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
   
    model = CatBoostRegressor(**catb_params_level1_one)
    model.fit(xtrain, ytrain, eval_set=[(xvalid, yvalid)], 
              early_stopping_rounds=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_3.csv", index=False)

0 0.7153175806447779
1 0.7229378511835743
2 0.7183815264772574
3 0.7179252839848066
4 0.7133229662622653
0.7175770417105364 0.0032468159435037154


In [ ]:
# catb_params_basic - With pred_1, 2, 3 - 0.7174672048416613 0.003167382534210746
# catb_params_level1_one - With pred_1, 2, 3 - 0.7174919521133101 0.003191406158001656
# catb_params_level1_one - With pred_1, 2, 3, 4, 5 - 0.7175770417105364 0.0032468159435037154

# Level 2 - Final predictions

In [ ]:
df = pd.read_csv("drive/MyDrive/input/30-days-of-ml/train_folds.csv")
df_test = pd.read_csv("drive/MyDrive/input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("drive/MyDrive/input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_1.csv")
df2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_2.csv")
df3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_train_pred_3.csv")

df_test1 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_1.csv")
df_test2 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_2.csv")
df_test3 = pd.read_csv("drive/MyDrive/input/30-days-of-ml/level1_test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.428013,8.446964,8.443256
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.392164,8.404102,8.411328
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.154991,8.166625,8.168323
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.407553,8.404738,8.400591
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.229510,8.238724,8.233623


In [ ]:
xgb_params_basic = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        # Remember to increase n_estimators to 10000 and increasing optuna trials to 200
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters            
        }

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
        
    # xgb_params_basic with early_stopping_rounds
    model = XGBRegressor(**xgb_params_basic, 
                         early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7216205433075689
1 0.7311051865296901
2 0.7245126039014183
3 0.7253661129601328
4 0.7194538357864458
0.7244116564970513 0.003952083522680557


### Hyperparameter tuning using Optuna

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
object_cols = [col for col in useful_features if 'cat' in col]

def run(trial):
    # We are optimizing in One 0th fold only
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features] 

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=1,
        objective='reg:squarederror',
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=10000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
# 50 trials for now, try with increasing it to 200
study.optimize(run, n_trials=50)

[I 2021-11-16 10:11:07,660] A new study created in memory with name: no-name-60efacaa-92b0-401c-b55a-4b557cca59c1


[0]	validation_0-rmse:7.65599
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[490]	validation_0-rmse:0.715302



[I 2021-11-16 10:11:09,552] Trial 0 finished with value: 0.7153023253836402 and parameters: {'learning_rate': 0.015466046429573594, 'reg_lambda': 0.07758025630572597, 'reg_alpha': 0.0026450532817366397, 'subsample': 0.6356608749952459, 'colsample_bytree': 0.41438950753117954, 'max_depth': 2}. Best is trial 0 with value: 0.7153023253836402.


[0]	validation_0-rmse:7.68731
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[626]	validation_0-rmse:0.715271



[I 2021-11-16 10:11:11,727] Trial 1 finished with value: 0.7152707361368491 and parameters: {'learning_rate': 0.011390536122624808, 'reg_lambda': 1.1435126474522713e-08, 'reg_alpha': 4.760615445704733, 'subsample': 0.11595597305864977, 'colsample_bytree': 0.19073418789771418, 'max_depth': 2}. Best is trial 1 with value: 0.7152707361368491.


...

[I 2021-11-16 10:12:49,875] Trial 47 finished with value: 0.7153501016919249 and parameters: {'learning_rate': 0.08849886553590262, 'reg_lambda': 1.379156258381984, 'reg_alpha': 0.018712718812086734, 'subsample': 0.2971839714628823, 'colsample_bytree': 0.13617479690914022, 'max_depth': 2}. Best is trial 34 with value: 0.7152533365936229.


[0]	validation_0-rmse:7.48479
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[171]	validation_0-rmse:0.715452



[I 2021-11-16 10:12:52,345] Trial 48 finished with value: 0.7154524560247199 and parameters: {'learning_rate': 0.03764651549762118, 'reg_lambda': 3.105133108915468, 'reg_alpha': 0.005302568348710179, 'subsample': 0.19901003934612524, 'colsample_bytree': 0.4890235322051588, 'max_depth': 7}. Best is trial 34 with value: 0.7152533365936229.


[0]	validation_0-rmse:7.56564
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[273]	validation_0-rmse:0.715345



[I 2021-11-16 10:12:53,781] Trial 49 finished with value: 0.71534540178321 and parameters: {'learning_rate': 0.027184905147299678, 'reg_lambda': 0.04725496226081904, 'reg_alpha': 6.65314221430684, 'subsample': 0.8771967726545317, 'colsample_bytree': 0.2899359131802528, 'max_depth': 2}. Best is trial 34 with value: 0.7152533365936229.


In [ ]:
study.best_trial

FrozenTrial(number=34, values=[0.7152533365936229], datetime_start=datetime.datetime(2021, 11, 16, 10, 12, 28, 215199), datetime_complete=datetime.datetime(2021, 11, 16, 10, 12, 29, 778575), params={'learning_rate': 0.026430193198644262, 'reg_lambda': 0.014668054268952402, 'reg_alpha': 0.07429590351843496, 'subsample': 0.101884043811053, 'colsample_bytree': 0.321900264653819, 'max_depth': 3}, distributions={'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'reg_lambda': LogUniformDistribution(high=100.0, low=1e-08), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'subsample': UniformDistribution(high=1.0, low=0.1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'max_depth': IntUniformDistribution(high=7, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=34, state=TrialState.COMPLETE, value=None)

In [ ]:
study.best_params

{'colsample_bytree': 0.321900264653819,
 'learning_rate': 0.026430193198644262,
 'max_depth': 3,
 'reg_alpha': 0.07429590351843496,
 'reg_lambda': 0.014668054268952402,
 'subsample': 0.101884043811053}

In [ ]:
# FrozenTrial(number=34, values=[0.7152533365936229], datetime_start=datetime.datetime(2021, 11, 16, 10, 12, 28, 215199), datetime_complete=datetime.datetime(2021, 11, 16, 10, 12, 29, 778575), params={'learning_rate': 0.026430193198644262, 'reg_lambda': 0.014668054268952402, 'reg_alpha': 0.07429590351843496, 'subsample': 0.101884043811053, 'colsample_bytree': 0.321900264653819, 'max_depth': 3}, distributions={'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'reg_lambda': LogUniformDistribution(high=100.0, low=1e-08), 'reg_alpha': LogUniformDistribution(high=100.0, low=1e-08), 'subsample': UniformDistribution(high=1.0, low=0.1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'max_depth': IntUniformDistribution(high=7, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=34, state=TrialState.COMPLETE, value=None)
# {'colsample_bytree': 0.321900264653819,
#  'learning_rate': 0.026430193198644262,
#  'max_depth': 3,
#  'reg_alpha': 0.07429590351843496,
#  'reg_lambda': 0.014668054268952402,
#  'subsample': 0.101884043811053}

In [ ]:
xgb_params_level2_one = {
        'random_state': 1,
        'objective': 'reg:squarederror',
        'n_jobs': -1,
         # Remember to increase n_estimators to 10000
        'n_estimators': 10000,

        # Only test with these parameters
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'tree_method': 'gpu_hist',
        # Only test with these parameters   
       
      'colsample_bytree': 0.321900264653819,
      'learning_rate': 0.026430193198644262,
      'max_depth': 3,
      'reg_alpha': 0.07429590351843496,
      'reg_lambda': 0.014668054268952402,
      'subsample': 0.101884043811053
      }


In [ ]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4,8.428013,8.446964,8.443256
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0,8.392164,8.404102,8.411328
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.154991,8.166625,8.168323
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1,8.407553,8.404738,8.400591
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0,8.229510,8.238724,8.233623


In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
        
    model = XGBRegressor(**xgb_params_level2_one,
                          early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    model.fit(xtrain, ytrain)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7162777997510507
1 0.7241882661194983
2 0.7195516245774057
3 0.7189806466642771
4 0.7144097799092993
0.7186816234043062 0.0033225425789825307


### Create submission 

In [ ]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("submission.csv", index=False)